In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
from scipy.sparse import coo_matrix
from Image import Image
import os

In [14]:
a = np.array([[1,2,3]])
Z0, W0 = np.polynomial.hermite.hermgauss(31)
W0.shape

(31,)